In [1]:
import pandas as pd

# Turn Off Pink Boxes 
import warnings
warnings.filterwarnings("ignore")

### 1. Load the tips dataset.



In [2]:
from pydataset import data
tips = data("tips")
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


#### a. Create a column named tip_percentage. This should be the tip amount divided by the total bill.


In [3]:
tips['tip_percentage'] = (tips.tip / tips.total_bill)
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


#### b. Create a column named price_per_person. This should be the total bill divided by the party size.


In [4]:
tips['price_per_person'] = round((tips.total_bill / tips['size']), 2)
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.49
2,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.45
3,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.00
4,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.84
5,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.15


#### c. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?


My prediction is that size of party, and time of day will play largely into tip amount.

#### d. Use select k best and recursive feature elimination to select the top 2 features for predicting tip amount. What are they?


In [5]:
X_train = tips.drop(columns= ['tip', 'sex', 'smoker','day', 'time'])
y_train = tips.tip

X_train.head()

,total_bill,size,tip_percentage,price_per_person
1,16.99,2,0.059447,8.49
2,10.34,3,0.160542,3.45
3,21.01,3,0.166587,7.00
4,23.68,2,0.139780,11.84
5,24.59,4,0.146808,6.15


In [6]:
from sklearn.feature_selection import SelectKBest, f_regression

# Since the question is asking for 2 features, I need to make sure my k is equal to 2

# Parameters: f_regression stats test, give me 2 features
f_selector = SelectKBest(f_regression, k=2)

# Find the top 2 X's correlated with y
f_selector.fit(X_train, y_train)

# Boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# Get list of top K features. 
f_feature = X_train.iloc[:,feature_mask].columns.tolist()

print("KBest Top Two Features:", f_feature)

KBest Top Two Features: ['total_bill', 'size']


#### e. Use select k best and recursive feature elimination to select the top 2 features for predicting tip percentage. What are they?


In [7]:
# Using the code from the curriculum:

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# initialize the ML algorithm
lm = LinearRegression()

# create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
rfe = RFE(lm, 2)

# fit the data using RFE
rfe.fit(X_train,y_train)  

# get the mask of the columns selected
feature_mask = rfe.support_

# get list of the column names. 
rfe_feature = X_train.iloc[:,feature_mask].columns.tolist()

print("Recursive Feature Elimination Top Two Features:",rfe_feature)

Recursive Feature Elimination Top Two Features: ['total_bill', 'tip_percentage']


In [8]:
# view list of columns and their ranking

# get the ranks
var_ranks = rfe.ranking_
# get the variable names
var_names = X_train.columns.tolist()
# combine ranks and names into a df for clean viewing
rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
# sort the df by rank
rfe_ranks_df.sort_values('Rank')

,Var,Rank
0,total_bill,1
2,tip_percentage,1
1,size,2
3,price_per_person,3


Takeaway: My most valuable feature is clearly total_bill, with size coming in second. Even though tip_percentage tied in the Recursive Feature Elimination, both models found size to be the next important feature. 

#### f. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

It's hard to tell in KBest or Recursive if my features are ordered by importance or column order. If it's in order of importance, then as K changes so does the importance of different variables. 

### 2. Write a function named `select_kbest` that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [9]:
def select_kbest(predictors, target, k_size):
    
    # Parameters: f_regression stats test
    f_selector = SelectKBest(f_regression, k= k_size)

    # Find the top predictor's correlated with target
    f_selector.fit(predictors, target)

    # Boolean mask of whether the column was selected or not. 
    feature_mask = f_selector.get_support()

    # Get list of top K features. 
    f_feature = predictors.iloc[:,feature_mask].columns.tolist()

    return f_feature

In [10]:
# Testing my new function
select_kbest(X_train, y_train, 2)

['total_bill', 'size']

### 3. Write a function named `select_rfe` that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [11]:
def select_rfe (predictors, target, k_size):
    # Initialize the ML algorithm
    lm = LinearRegression()

    # Create the rfe object, indicating the ML object (lm) and the number of features I want to end up with. 
    rfe = RFE(lm, k_size)

    # Fit the data using RFE
    rfe.fit(predictors,target)  

    # Get the mask of the columns selected
    feature_mask = rfe.support_

    # Get list of the column names. 
    rfe_feature = predictors.iloc[:,feature_mask].columns.tolist()
    
    # view list of columns and their ranking

    # get the ranks
    var_ranks = rfe.ranking_
    # get the variable names
    var_names = predictors.columns.tolist()
    # combine ranks and names into a df for clean viewing
    rfe_ranks_df = pd.DataFrame({'Var': var_names, 'Rank': var_ranks})
    # sort the df by rank
    rfe_ranks_df.sort_values('Rank')
    
    print(rfe_feature)
    return  rfe_ranks_df.sort_values('Rank')

In [12]:
# Testing my new function
select_rfe(X_train, y_train, 2)

['total_bill', 'tip_percentage']


,Var,Rank
0,total_bill,1
2,tip_percentage,1
1,size,2
3,price_per_person,3


### 4. Load the `swiss` dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [13]:
swiss = data("swiss")
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [14]:
# Separating predictors from target
x = swiss.drop(columns= ['Fertility'])
y = swiss.Fertility

In [19]:
select_kbest(x, y, 3)

['Examination', 'Education', 'Catholic']

In [16]:
select_rfe(x, y, 3)

['Examination', 'Education', 'Infant.Mortality']


,Var,Rank
1,Examination,1
2,Education,1
4,Infant.Mortality,1
0,Agriculture,2
3,Catholic,3


Takeaway: The models show Examination, Education, and Infant.Mortality as the top three features for the swiss dataset. This becomes more apparent when you adjust the k of kbest to 4.